### Importing LIB

In [1]:
import sys 
!{sys.executable} -m pip install numpy pandas tensorflow scikit-learn imblearn
import os
import time
import shutil
import itertools
import sklearn
import cv2
import numpy as np 
import pandas as pd
import seaborn as sns
import imblearn
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
import tensorflow as ts
from tensorflow import keras
from tensorflow.keras.layers import Conv2D ,MaxPooling2D , Flatten , Dense , Activation , Dropout , BatchNormalization
from tensorflow.keras.models import Model , load_model , Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam , Adamax
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import categorical_crossentropy

import warnings

warnings.filterwarnings("ignore")


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\gabri\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Quick analysis of the data

In [2]:
data= "archive\hmnist_28_28_RGB.csv"

data = pd.read_csv(data)

data.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2


### Separating the Target variable 

In machine learning, for an obvious reason, you need to separate the target variable before training the model on it. This ensures that the model learns patterns from the input features rather than memorizing the target values.

In [3]:
Label = data['label']

Data = data.drop(columns =['label'] )


### Reshaping the data for an image of 28×28

In [4]:

from imblearn.over_sampling import RandomOverSampler  
from imblearn.under_sampling import RandomUnderSampler

Data = np.array(Data).reshape(-1 ,28,28,3)

print("Shape Data :", Data.shape)

Shape Data : (10015, 28, 28, 3)


In [5]:
Label = np.array(Label)

Label


array([2, 2, 2, ..., 0, 0, 6], dtype=int64)

### Putting labels for the different types of skin diseases

In [6]:
classes = {
4   : {'nv' : 'melanocytic nevi'},
6   : {"mel": 'melanoma'},
2   : {"bkl": 'benign keratosis-like lesions'},
1   : {"bcc": 'basal cell carcinoma'},
5   : {"vasc":"pyogenic granulomas and hemorrhage"},
0   : {"akiec" : "Actinic keratoses ans intraepithelial carcinomae"},
3   : {"df"  : "dermatofibroma"}


}
 

### Creation of the Train/Test dataset variables

In [9]:
from sklearn.model_selection import train_test_split



X_train , X_test , y_train , y_test = train_test_split(Data ,
                                                        Label ,
                                                        test_size= 0.3,
                                                        random_state=23) 

### Fixing the Class Imbalance in the Dataset

For better results, it is recommended to have a balanced number of samples across different classes in the dataset. To address class imbalance, we can use under-sampling or over-sampling techniques.

Under-sampling: Reduces the number of samples in the majority class to match the minority class.

Over-sampling: Increases the number of samples in the minority class by duplicating or generating new samples (e.g., using SMOTE).

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE

smote= SMOTE()

# Flatten the 4D data 
X_train_flat = X_train.reshape(X_train.shape[0], -1)

# Apply SMOTE on the flattened data
X_train_res_flat, y_train_res = smote.fit_resample(X_train_flat , y_train)

# Reshape X_train_res_flat back to 4D
X_train_res = X_train_res_flat.reshape(X_train_res_flat.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3])

# Check the new distribution of the training data
print(f"Original dataset shape: {Counter(y_train)}")
print(f"Resampled dataset shape: {Counter(y_train_res)}")


In [14]:
print(f'X_train shape :{X_train.shape} \n X_test shape: {X_test}')

print(f'y_train shape :{y_train.shape} \n y_test shape: {y_test}')

X_train shape :(7010, 28, 28, 3) 
 X_test shape: [[[[235 164 178]
   [225 154 166]
   [232 155 169]
   ...
   [213 119 149]
   [194 108 140]
   [194 110 143]]

  [[235 170 181]
   [223 154 165]
   [233 158 171]
   ...
   [200 106 139]
   [193 106 138]
   [196 110 142]]

  [[235 169 179]
   [232 161 172]
   [233 163 174]
   ...
   [189 101 136]
   [205 116 149]
   [196 108 144]]

  ...

  [[237 185 186]
   [237 184 183]
   [235 177 175]
   ...
   [221 146 155]
   [213 132 147]
   [212 131 149]]

  [[238 184 187]
   [237 181 183]
   [238 183 179]
   ...
   [217 136 148]
   [214 134 146]
   [215 135 151]]

  [[235 172 176]
   [232 167 170]
   [227 164 159]
   ...
   [218 138 151]
   [217 140 153]
   [216 141 156]]]


 [[[154 125 152]
   [159 129 157]
   [160 132 161]
   ...
   [151 120 150]
   [150 120 149]
   [148 118 146]]

  [[157 128 157]
   [160 130 158]
   [162 133 162]
   ...
   [154 124 155]
   [153 124 154]
   [150 121 151]]

  [[158 128 153]
   [162 133 159]
   [162 133 159]
   

### Encode the variables in categorical type

In [15]:


from keras.utils import to_categorical

y_train = to_categorical(y_train)

y_test =  to_categorical(y_test)

In [16]:
datagen = ImageDataGenerator(rescale=( 1./255),
                            rotation_range = 10,
                            zoom_range = 0.1,
                            width_shift_range = 0.1,
                            height_shift_range = 0.1)

testgen = ImageDataGenerator(rescale=( 1./255))
                            

### Setting up the learning rate reduction function

In [35]:
from keras.callbacks import ReduceLROnPlateau


learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                                patience = 2,
                                                verbose = 1,
                                                factor =  0.5,
                                                min_lr = 0.0001)



### Creation of the model

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, ReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# 🔹 Création du modèle
model = Sequential()

# Input
model.add(Input(shape=(28, 28, 3)))

# Block 1
model.add(Conv2D(32, (3,3), padding='same', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D())

# Block 2
model.add(Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D())

# Block 3
model.add(Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D())

# Flatten + Fully Connected Layers
model.add(Flatten())

model.add(Dropout(0.6))  
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.L2(0.0005)))
model.add(Dense(128, activation='relu'))

model.add(Dense(7, activation='softmax', kernel_initializer='glorot_uniform'))


model.compile(optimizer=Adam(learning_rate=0.0003, decay=1e-5), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)




In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs=50,  # On augmente un peu les epochs pour plus de précision
                    batch_size=128,
                    validation_data=(X_test, y_test),
                    callbacks=[lr_scheduler, early_stopping])


                        

### Output the result

In [38]:
train_score = model.evaluate(X_train, y_train , verbose = 1)
test_score = model.evaluate(X_test , y_test , verbose =  1)


print("Train Loss :", train_score[0])
print("Train_accuracy : ",train_score[1])


print("Test Loss : " , test_score[0])
print('Test accuracy : ', test_score[1])


220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8468 - loss: 0.5656
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7781 - loss: 0.7792
Train Loss : 0.5543563365936279
Train_accuracy :  0.8526390790939331
Test Loss :  0.7589062452316284
Test accuracy :  0.781364381313324


### Quick conclusion

As we can see, the model can detect whether a spot on your skin is malignant within an acceptable range. However, there is room for improvement in the model to reduce the significant loss during training.

In [40]:
model.save('model_optimized.keras')


In [41]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the optimized model
model = load_model('model_optimized.keras')

# Evaluate the model on the test set
test_score = model.evaluate(X_test, y_test, verbose=1)

# Display the test loss and accuracy
test_loss, test_accuracy = test_score
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7781 - loss: 0.7792
Test Loss: 0.7589062452316284
Test Accuracy: 0.781364381313324
